In [1]:
import pymysql
import pandas as pd
import pickle
from sqlalchemy import create_engine
import sys
import json
import numpy as np
from pandas.io.json import json_normalize 
import string
import os
from calculate_retention import CalculateRetention
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# у нас есть несколько json-файлов. читаем их и объединяем в один датафрейм
# 7 файлов - информация о посещениях клиентами ресурса в течение 7 дней. 
# Считаем retension rate 7ого дня 
base_path = '../ga_sessions'
frames = []
for filename in os.listdir(base_path):
    if filename.startswith('ga_sessions'):
        with open(os.path.join(base_path, filename)) as handler:
            sessions = []
            for line in handler.readlines():
                session = json.loads(line)
                sessions.append(session)
            print(len(sessions))
        frames.append(json_normalize(sessions).drop(['customDimensions','hits'],axis='columns'))
df = pd.concat(frames)

ga_sessions_20170726.json
2725
ga_sessions_20170727.json
2529
ga_sessions_20170728.json
2433
ga_sessions_20170729.json
1597
ga_sessions_20170730.json
1799
ga_sessions_20170731.json
2620
ga_sessions_20170801.json
2556


In [138]:
# для расчета коэффициента удержания нам достаточно поля с id клиента и даты посещения 
df1 = df.loc[:, 'date':'fullVisitorId']
df1['date'] = pd.to_datetime(df1['date'], format = '%Y%m%d')
df1 = df1.rename(columns={'fullVisitorId': 'id'})
df1.head()

,date,id
0,2017-07-26,4576978993060509058
1,2017-07-26,0686272241135131743
2,2017-07-26,5769691250560076510
3,2017-07-26,2341709029923823417
4,2017-07-26,0851060264696052177


In [139]:
# используем id клиента в качестве id строки.
# Группируем по id клиента и получаем первую дату посещения клиента 
df1.set_index('id', inplace=True)
df1['first_time'] = df1.groupby(level=0)['date'].min().apply(lambda x: x)
df1

,date,first_time
id,,
4576978993060509058,2017-07-26,2017-07-26
0686272241135131743,2017-07-26,2017-07-26
5769691250560076510,2017-07-26,2017-07-26
2341709029923823417,2017-07-26,2017-07-26
0851060264696052177,2017-07-26,2017-07-26
...,...,...
8930667883681133792,2017-08-01,2017-08-01
3723888941430883184,2017-08-01,2017-08-01
9082216468378814100,2017-08-01,2017-08-01


In [140]:
df1.reset_index(inplace=True)
df1.head()

,id,date,first_time
0,4576978993060509058,2017-07-26,2017-07-26
1,0686272241135131743,2017-07-26,2017-07-26
2,5769691250560076510,2017-07-26,2017-07-26
3,2341709029923823417,2017-07-26,2017-07-26
4,0851060264696052177,2017-07-26,2017-07-26


In [156]:
# группируем данные по первому дню посещения, а также по дням недели 
# и считаем количество уникальных пользователей каждый день
grouped = df1.groupby(['first_time', 'date'])
cohorts = grouped.agg({'id': pd.Series.nunique})
cohorts

id
first_time date            
2017-07-26 2017-07-26  2443
           2017-07-27   141
           2017-07-28    60
           2017-07-29    36
           2017-07-30    34
           2017-07-31    69
           2017-08-01    41
2017-07-27 2017-07-27  2156
           2017-07-28    94
           2017-07-29    28
           2017-07-30    31
           2017-07-31    53
           2017-08-01    33
2017-07-28 2017-07-28  2079
           2017-07-29    45
           2017-07-30    29
           2017-07-31    46
           2017-08-01    29
2017-07-29 2017-07-29  1389
           2017-07-30    48
           2017-07-31    21
           2017-08-01     9
2017-07-30 2017-07-30  1507
           2017-07-31    52
           2017-08-01    20
2017-07-31 2017-07-31  2127
           2017-08-01    93
2017-08-01 2017-08-01  2068

In [157]:
# нумеруем дни недели 
def timedelta(df):
    df['timedelta'] = np.arange(len(df)) + 1
    return df

cohorts = cohorts.groupby(level=0).apply(timedelta)
cohorts

id  timedelta
first_time date                       
2017-07-26 2017-07-26  2443          1
           2017-07-27   141          2
           2017-07-28    60          3
           2017-07-29    36          4
           2017-07-30    34          5
           2017-07-31    69          6
           2017-08-01    41          7
2017-07-27 2017-07-27  2156          1
           2017-07-28    94          2
           2017-07-29    28          3
           2017-07-30    31          4
           2017-07-31    53          5
           2017-08-01    33          6
2017-07-28 2017-07-28  2079          1
           2017-07-29    45          2
           2017-07-30    29          3
           2017-07-31    46          4
           2017-08-01    29          5
2017-07-29 2017-07-29  1389          1
           2017-07-30    48          2
           2017-07-31    21          3
           2017-08-01     9          4
2017-07-30 2017-07-30  1507          1
           2017-07-31    52          2
           2017-08-01    20          3
2017-07-31 2017-07-31  2127          1
           2017-08-01    93          2
2017-08-01 2017-08-01  2068          1

In [158]:
cohorts.reset_index(inplace=True)
cohorts

,first_time,date,id,timedelta
0,2017-07-26,2017-07-26,2443,1
1,2017-07-26,2017-07-27,141,2
2,2017-07-26,2017-07-28,60,3
3,2017-07-26,2017-07-29,36,4
4,2017-07-26,2017-07-30,34,5
5,2017-07-26,2017-07-31,69,6
6,2017-07-26,2017-08-01,41,7
7,2017-07-27,2017-07-27,2156,1
8,2017-07-27,2017-07-28,94,2
9,2017-07-27,2017-07-29,28,3


In [163]:
cohorts = cohorts.drop(columns=['date'], axis=1)
cohorts

,first_time,id,timedelta
0,2017-07-26,2443,1
1,2017-07-26,141,2
2,2017-07-26,60,3
3,2017-07-26,36,4
4,2017-07-26,34,5
5,2017-07-26,69,6
6,2017-07-26,41,7
7,2017-07-27,2156,1
8,2017-07-27,94,2
9,2017-07-27,28,3


In [164]:
cohorts.set_index(['first_time', 'timedelta'], inplace=True)
cohorts

id
first_time timedelta      
2017-07-26 1          2443
           2           141
           3            60
           4            36
           5            34
           6            69
           7            41
2017-07-27 1          2156
           2            94
           3            28
           4            31
           5            53
           6            33
2017-07-28 1          2079
           2            45
           3            29
           4            46
           5            29
2017-07-29 1          1389
           2            48
           3            21
           4             9
2017-07-30 1          1507
           2            52
           3            20
2017-07-31 1          2127
           2            93
2017-08-01 1          2068

In [165]:
# количество первых посещений клиентов под дням недели 
cohort_group_size = cohorts['id'].groupby(level=0).first()
cohort_group_size.head()

first_time
2017-07-26    2443
2017-07-27    2156
2017-07-28    2079
2017-07-29    1389
2017-07-30    1507
Name: id, dtype: int64

In [166]:
# возвращаем датафрейм с новым уровнем меток столбцов по дате посещения
cohorts['id'].unstack(0)

first_time,2017-07-26,2017-07-27,2017-07-28,2017-07-29,2017-07-30,2017-07-31,2017-08-01
timedelta,,,,,,,
1,2443.0,2156.0,2079.0,1389.0,1507.0,2127.0,2068.0
2,141.0,94.0,45.0,48.0,52.0,93.0,NaN
3,60.0,28.0,29.0,21.0,20.0,NaN,NaN
4,36.0,31.0,46.0,9.0,NaN,NaN,NaN
5,34.0,53.0,29.0,NaN,NaN,NaN,NaN
6,69.0,33.0,NaN,NaN,NaN,NaN,NaN
7,41.0,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
# делим количество посещений уникальных пользователей N-ого дня на количество посещений 1ого дня
user_retention = cohorts['id'].unstack(0).divide(cohort_group_size, axis=1)
user_retention

first_time,2017-07-26,2017-07-27,2017-07-28,2017-07-29,2017-07-30,2017-07-31,2017-08-01
timedelta,,,,,,,
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
2,0.057716,0.043599,0.021645,0.034557,0.034506,0.043724,NaN
3,0.024560,0.012987,0.013949,0.015119,0.013271,NaN,NaN
4,0.014736,0.014378,0.022126,0.006479,NaN,NaN,NaN
5,0.013917,0.024583,0.013949,NaN,NaN,NaN,NaN
6,0.028244,0.015306,NaN,NaN,NaN,NaN,NaN
7,0.016783,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
user_retention = round(user_retention.T, 2)
user_retention

timedelta,1,2,3,4,5,6,7
first_time,,,,,,,
2017-07-26,1.0,0.06,0.02,0.01,0.01,0.03,0.02
2017-07-27,1.0,0.04,0.01,0.01,0.02,0.02,NaN
2017-07-28,1.0,0.02,0.01,0.02,0.01,NaN,NaN
2017-07-29,1.0,0.03,0.02,0.01,NaN,NaN,NaN
2017-07-30,1.0,0.03,0.01,NaN,NaN,NaN,NaN
2017-07-31,1.0,0.04,NaN,NaN,NaN,NaN,NaN
2017-08-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN
